## Learning to rank

Требуется реализовать алгоритм машинного обучения ранжированию и с его помощью выбрать 5 наиболее релевантных документов, отсортировать их по убыванию релевантности.

In [1]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.datasets import load_svmlight_file
from collections import defaultdict

In [2]:
X, y, query_ids = load_svmlight_file('l2r/train.txt', query_id=True)

https://logic.pdmi.ras.ru/~sergey/teaching/mlspsu17/21-learningtorank.pdf

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.180.634&rep=rep1&type=pdf

1. каждая строка файла соответствует номеру документа

2. query_ids - qid для кажой строки соответствует номеру запроса

3. y - каждой паре (doc_id, query_id) соответствует число - рейтинг документа по запросу (правильная метка) 

4. X - разреженная матрица признаков для пар (doc_id, query_id)

### Метрики ранжирования

$$ DCG_k = \sum_{i=1}^{k} \frac{2^{r_i} - 1}{\log_2(1 + i)}$$

$$ NDSG_k = \frac{DCG_k}{IDCG_k} $$

$r_i$ - рейтинг документа на позиции i

$k$ - число документов

$IDCG_k$ - значение $DCG_k$ при ранжировании по истинным значениям

### LambdaMART

$x_i$ - векторы признаков

$y_i$ - правильные метки

$s_i = F(x_i)$ - предсказание модели (рейтинг документа)

$s_{ij} = s_i - s_j = F(x_i) - F(x_j)$

$y_{ij} = y_i - y_j$

$C_{ij} = C(s_{ij}) = s_i - s_j + \log(1 + e^{s_i - s_j})$ - функция ошибки

$\frac{\partial{C_{ij}}}{\partial{s_{ij}}} = - \frac{1}{1 + e^{s_{ij}}}$

$\lambda_{ij} = y_{ij}|\Delta NDCG \frac{\partial{C_{ij}}}{\partial{s_{ij}}}|$

$\Delta NDCG$ - изменение $NDCG$ при обмене позиций документов с номерами i и j  

$grad_i = \lambda_i = \sum_{j} \lambda_{ij} = \sum_{j} y_{ij}|\Delta NDCG \frac{1}{1 + e^{s_{ij}}}|$ - градиенты

$hess_i = \frac{\partial{y_i}}{\partial{F(x_i)}} = \sum_{j}|\Delta NDCG \frac{1}{1 + e^{s_{ij}}} (1 - \frac{1}{1 + e^{s_{ij}}})|$ - вторые производные

In [3]:
class LambdaMART:
    def __init__(self, X, y, query_ids, mode="train"):
        self.X = X
        self.y = y
        self.query_ids = query_ids
        self.query2docs = defaultdict(list)
        
        for doc_id, query_id in enumerate(query_ids):
            self.query2docs[query_id].append(doc_id)
        
        if mode == 'train':
            self.query2idcg_5 = dict()
            for query_id in self.query2docs:
                self.query2idcg_5[query_id] = self.IDCG_k(self.y[self.query2docs[query_id]], k=5)
            self.query_y_ij = dict()
            for query_id in self.query2docs:
                docs = self.query2docs[query_id]
                y = self.y[docs]
                y_ij = np.zeros((len(docs), len(docs)))
                y_ij += (y.reshape(-1, 1) > y).astype(int)
                y_ij -= (y.reshape(-1, 1) < y).astype(int)
                self.query_y_ij[query_id] = y_ij
    
    @staticmethod
    def IDCG_k(y, k=None):
        sorted_y = np.sort(y)[::-1]
        if k:
            size = y.shape[0] if y.shape[0] < k else k
            sorted_y = sorted_y[:size]
        idcg = np.sum((2.0 ** sorted_y - 1) / np.log(np.arange(1, sorted_y.shape[0] + 1) + 1))
        if np.isclose(idcg, 0.0):
            return 1.0
        return idcg
    
    def objective(self, y_true, y_pred):
        grad = np.zeros(self.y.shape[0])
        hess = np.zeros(self.y.shape[0])
        for query_id in self.query2docs:
            docs = np.array(self.query2docs[query_id])
            s_i = y_pred[docs]
            s_ij = np.abs(s_i.reshape(-1, 1) - s_i)
            s_ij[s_ij > 50] = 50
            dC_ds = 1.0 / (1 + np.exp(s_ij)) # without sign
    
            sorted_ids = np.argsort(s_i)[::-1]
            sorted_ids_mask = (sorted_ids < 5).astype(int)
            idcg_5 = self.query2idcg_5[query_id]
            y_i = self.y[docs]
            y_ij = self.query_y_ij[query_id]
            
            delta_ndcg = np.abs((2 ** y_i.reshape(-1, 1) - 2 ** y_i)
                * ((sorted_ids_mask.reshape(-1, 1) / (np.log(sorted_ids.reshape(-1, 1) + 2)))
                   - (sorted_ids_mask / (np.log(sorted_ids + 2))))) / idcg_5
            
            grad[docs] = -np.sum(y_ij * delta_ndcg * dC_ds, axis=1) # sign
            hess[docs] = np.sum(delta_ndcg * dC_ds * (1 - dC_ds), axis=1)
            hess[np.isclose(hess, 0.0)] = 1.0
        return grad, hess

In [4]:
train = LambdaMART(X, y, query_ids)

In [5]:
%%time
params = {'objective': train.objective, 'max_depth': 5, 'nthread': -1, 'n_estimators': 500}
model = XGBRegressor(**params)
model.fit(X, y)

CPU times: user 5h 56min 16s, sys: 4min 30s, total: 6h 46s
Wall time: 12h 17min 21s


XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             objective=<bound method LambdaMART.objective of <__main__.LambdaMART object at 0x7faa268e5890>>,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method=None, validate_parameters=False,
             verbosity=None)

In [6]:
X_test, y_test, query_ids_test = load_svmlight_file('l2r/test.txt', query_id=True)
test = LambdaMART(X_test, y_test, query_ids_test, mode="test")
y_pred = model.predict(test.X)

In [7]:
with open("submission.txt", 'w') as write_file:
    print("QueryId,DocumentId", file=write_file)
    for query_id in test.query2docs:
        docs = test.query2docs[query_id]
        y_pred_i = y_pred[docs]
        sorted_ids = np.argsort(y_pred_i)[::-1]
        sorted_docs = np.array(docs)[sorted_ids]
        for doc_id in sorted_docs:
            print(f"{query_id},{doc_id+1}", file=write_file)